45

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 字體設定
system = platform.system()
if system == 'Darwin':
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC', 'Heiti TC', 'sans-serif']
elif system == 'Windows':
    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'sans-serif']
else:
    plt.rcParams['font.sans-serif'] = ['Noto Sans CJK TC', 'WenQuanYi Micro Hei', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

# 1. 篩選出對手在關鍵分（18分以上）殺球且周天成為擊球者的數據
opponent_smash = df[(df['opponent_score'] >= 18) & (df['player'] == 'CHOU Tien Chen') & (df['opponent_type'] == 4.0)].copy()

# 2. 計算周天成各種回球類型的次數和比例
shot_counts = opponent_smash['type'].value_counts()
shot_percentages = opponent_smash['type'].value_counts(normalize=True) * 100

# 3. 計算各種回球方式的失分率
loss_rates = {}
for shot_type in shot_counts.index:
    # 找到所有該回球類型的rally_id
    shot_rally_ids = opponent_smash[opponent_smash['type'] == shot_type]['rally_id'].unique()

    # 初始化失分次數
    losses = 0

    # 遍歷每個rally_id
    for rally_id in shot_rally_ids:
        # 篩選出當前rally_id中，周天成的所有擊球
        rally_shots = df[(df['rally_id'] == rally_id) & (df['player'] == 'CHOU Tien Chen')]

        # 找到最後一次擊球
        last_shot = rally_shots.iloc[-1]

        # 檢查該次擊球是否有失分原因
        if pd.notna(last_shot['lose_reason']):
            losses += 1

    # 計算失分率
    if shot_counts[shot_type] > 0:
        loss_rate = (losses / shot_counts[shot_type]) * 100
    else:
        loss_rate = 0
    loss_rates[shot_type] = loss_rate

# 4. 創建一個DataFrame來呈現結果
results_df = pd.DataFrame({
    '回球類型': shot_counts.index,
    '次數': shot_counts.values,
    '比例 (%)': shot_percentages.values,
    '失分率 (%)': [loss_rates[shot] for shot in shot_counts.index]
})

print(results_df)

# 5. 可視化回球類型分佈（長條圖）
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']
fig, ax = plt.subplots(figsize=(12, 7))
bars = ax.bar(results_df['回球類型'], results_df['次數'], color=colors, alpha=0.8)

# 在柱子上方標註數值
for i, v in enumerate(results_df['次數']):
    ax.text(i, v + max(results_df['次數'])*0.01, str(int(v)),
            ha='center', va='bottom', fontsize=10)

ax.set_title('周天成在對手關鍵分殺球時的回球類型分佈', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('回球類型', fontsize=12)
ax.set_ylabel('次數', fontsize=12)
plt.xticks(rotation=45, ha='right')
ax.grid(True, alpha=0.3, linestyle='--', axis='y')
plt.tight_layout()

46

In [ ]:
import pandas as pd

# 定義防守和進攻狀態的球種
defensive_types = ['接殺防守', '網前球']
offensive_types = ['殺球', '推撲球']

# 篩選出周天成擊球的數據
chou_data = df[df['player'] == 'CHOU Tien Chen'].copy()

# 初始化計數器
successful_turnarounds = 0

# 遍歷每個match_id和set
for match_id in chou_data['match_id'].unique():
    for set_num in chou_data[chou_data['match_id'] == match_id]['set'].unique():
        # 篩選出當前match_id和set的數據
        set_data = chou_data[(chou_data['match_id'] == match_id) & (chou_data['set'] == set_num)].copy()

        # 遍歷每個rally
        for rally in set_data['rally'].unique():
            rally_data = set_data[set_data['rally'] == rally].copy()

            # 找到所有連續三拍或三拍以上的回合
            if len(rally_data) >= 3:
                for i in range(len(rally_data) - 2):
                    # 檢查前兩拍是否為防守狀態
                    if (rally_data.iloc[i]['type'] in defensive_types) and (rally_data.iloc[i+1]['type'] in defensive_types):
                        # 檢查第三拍是否為進攻狀態
                        if rally_data.iloc[i+2]['type'] in offensive_types:
                            # 檢查該球局是否由周天成得分
                            last_shot = df[(df['match_id'] == match_id) & (df['set'] == set_num) & (df['rally'] == rally)]['getpoint_player'].iloc[-1]
                            if last_shot == 'CHOU Tien Chen':
                                successful_turnarounds += 1
                                break  # 每個rally只計一次

# 輸出結果
print(f"周天成在連續三拍或三拍以上的回合中，先防守後進攻並最終得分的球局數量：{successful_turnarounds}")

47

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 字體設定
system = platform.system()
if system == 'Darwin':
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC', 'Heiti TC', 'sans-serif']
elif system == 'Windows':
    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'sans-serif']
else:
    plt.rcParams['font.sans-serif'] = ['Noto Sans CJK TC', 'WenQuanYi Micro Hei', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

# 1. 篩選出 'player' 欄位為 'CHOU Tien Chen' 的所有回合
chou_rounds = df[df['player'] == 'CHOU Tien Chen'].copy()

# 2. 計算「得分」回合的平均拍數
scoring_rounds = chou_rounds[chou_rounds['getpoint_player'] == 'CHOU Tien Chen']
if not scoring_rounds.empty:
    avg_scoring_rounds = scoring_rounds['ball_round'].mean()
    print(f"周天成得分回合的平均拍數：{avg_scoring_rounds:.2f}")
else:
    avg_scoring_rounds = 0
    print("沒有找到周天成得分回合的數據。")

# 3. 計算「失分」回合的平均拍數
losing_rounds = chou_rounds[(chou_rounds['getpoint_player'] != 'CHOU Tien Chen') & (chou_rounds['getpoint_player'].notna())]
if not losing_rounds.empty:
    avg_losing_rounds = losing_rounds['ball_round'].mean()
    print(f"周天成失分回合的平均拍數：{avg_losing_rounds:.2f}")
else:
    avg_losing_rounds = 0
    print("沒有找到周天成失分回合的數據。")

# 4. 比較這兩個平均拍數
if avg_scoring_rounds > 0 or avg_losing_rounds > 0:
    if avg_scoring_rounds > avg_losing_rounds:
        comparison_result = "周天成得分回合的平均拍數較高。"
    elif avg_scoring_rounds < avg_losing_rounds:
        comparison_result = "周天成失分回合的平均拍數較高。"
    else:
        comparison_result = "周天成得分回合和失分回合的平均拍數相同。"
    print(comparison_result)
else:
    comparison_result = "沒有找到足夠的數據來比較周天成得分和失分回合的平均拍數。"

# 5. 將結果儲存在變數中
results = {
    "avg_scoring_rounds": avg_scoring_rounds,
    "avg_losing_rounds": avg_losing_rounds,
    "comparison_result": comparison_result
}

# 可视化呈現
labels = ['得分回合', '失分回合']
avg_round_nums = [results['avg_scoring_rounds'], results['avg_losing_rounds']]
colors = ['#2A9D8F', '#E63946']  # 綠色代表得分，紅色代表失分

fig, ax = plt.subplots(figsize=(8, 6))
bars = ax.bar(labels, avg_round_nums, color=colors)

# 在柱子上方標註數值
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, f'{yval:.2f}', ha='center', va='bottom')

ax.set_ylabel('平均拍數', fontsize=12)
ax.set_title('周天成得分與失分回合的平均拍數比較', fontsize=16, fontweight='bold')
ax.grid(axis='y', linestyle='--')

plt.tight_layout()

48

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# 字體設定
system = platform.system()
if system == 'Darwin':
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC', 'Heiti TC', 'sans-serif']
elif system == 'Windows':
    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'sans-serif']
else:
    plt.rcParams['font.sans-serif'] = ['Noto Sans CJK TC', 'WenQuanYi Micro Hei', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

# 篩選 'player' 欄位為 'CHOU Tien Chen' 且 'lose_reason' 欄位不為空的資料
chou_errors = df[(df['player'] == 'CHOU Tien Chen') & (df['lose_reason'].notna())]

# 建立固定尺寸的圖表
fig, ax = plt.subplots(figsize=(12, 7))

# 繪製散佈圖
if not chou_errors.empty:
    sns.scatterplot(x='landing_x', y='landing_y', data=chou_errors, ax=ax, color='#FF6B6B', alpha=0.5)

    # 設定標題和標籤
    ax.set_title('周天成失誤球落點分佈', fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('落點 X 座標', fontsize=12)
    ax.set_ylabel('落點 Y 座標', fontsize=12)

    # 加入網格線
    ax.grid(True, alpha=0.3, linestyle='--')

    # 調整布局避免文字被裁切
    plt.tight_layout()
else:
    ax.text(0.5, 0.5, '找不到周天成的失誤球數據', ha='center', va='center', fontsize=12)
    ax.axis('off')

In [ ]:
49

In [ ]:
import platform
import matplotlib.pyplot as plt
import pandas as pd

# 字體設定
system = platform.system()
if system == 'Darwin':
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang TC', 'Heiti TC', 'sans-serif']
elif system == 'Windows':
    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'sans-serif']
else:
    plt.rcParams['font.sans-serif'] = ['Noto Sans CJK TC', 'WenQuanYi Micro Hei', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

# 篩選 'player' 欄位為 'CHOU Tien Chen' 的資料
chou_data = df[df['player'] == 'CHOU Tien Chen']

# 篩選出對手得分的資料 (getpoint_player 不是 CHOU Tien Chen)
opponent_getpoint = df[df['getpoint_player'] != 'CHOU Tien Chen']

# 合併條件，找出周天成的失分原因
chou_lose_reasons = opponent_getpoint[opponent_getpoint['player'] == 'CHOU Tien Chen']['lose_reason'].dropna()

# 統計 'lose_reason' 欄位中每個原因出現的次數
lose_reason_counts = chou_lose_reasons.value_counts()

# 建立固定尺寸的圖表
fig, ax = plt.subplots(figsize=(12, 7))

# 圓餅圖專用色系（對比度高）
colors = ['#E63946', '#2A9D8F', '#F4A261', '#264653', '#E76F51', '#8338EC', '#06BCC1', '#FF6B35']

if not lose_reason_counts.empty:
    # 繪製圓餅圖
    wedges, texts, autotexts = ax.pie(lose_reason_counts, autopct='%1.1f%%', startangle=90, colors=colors, pctdistance=0.85)

    # 設定圓餅圖樣式
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')

    # 設定標題
    ax.set_title('周天成失分原因分析', fontsize=16, fontweight='bold')

    # 創建圖例放在圓餅圖旁邊，不使用 labels 參數
    ax.legend(wedges, lose_reason_counts.index, loc='center left', bbox_to_anchor=(1, 0, 0.5, 1), fontsize=11)

    # 調整布局
    plt.tight_layout()
else:
    ax.text(0.5, 0.5, '找不到周天成的失分原因數據', ha='center', va='center', fontsize=12)
    ax.axis('off')
    plt.tight_layout()

print(lose_reason_counts)

50

In [ ]:
import pandas as pd

# 篩選 'getpoint_player' 欄位為 'CHOU Tien Chen' 的所有資料
chou_getpoint = df[df['getpoint_player'] == 'CHOU Tien Chen']

# 計算篩選後資料中 'ball_round' 欄位的平均值
if not chou_getpoint.empty:
    average_ball_round = chou_getpoint['ball_round'].mean()

    print(f"周天成得分回合的平均擊球數: {average_ball_round:.2f}")

    # 判斷周天成是傾向於短拍數得分還是長拍數得分
    if average_ball_round < 5:
        tendency = "短拍數得分"
    elif average_ball_round < 10:
        tendency = "中等拍數得分"
    else:
        tendency = "長拍數得分"

    print(f"周天成傾向於: {tendency}")
else:
    average_ball_round = None
    print("找不到周天成得分的資料")